In [43]:
import nemo
from nemo.collections.tts.models import FastPitchModel
from nemo.collections.tts.models import HifiGanModel
import tensorrt as trt
import torch
import onnx
from loguru import logger
import onnxruntime as ort
import numpy as np
from time import perf_counter
import pycuda.autoinit
import pycuda.driver as cuda

ModuleNotFoundError: No module named 'pycuda'

In [11]:
# Instantiate the model
fastpitch = FastPitchModel.from_pretrained("tts_en_fastpitch").eval().cuda()
hifigan = HifiGanModel.from_pretrained("tts_en_hifitts_hifigan_ft_fastpitch").eval().cuda()

[NeMo I 2024-11-03 14:53:52 cloud:58] Found existing object /home/antixk/.cache/torch/NeMo/NeMo_1.23.0/tts_en_fastpitch_align/b7d086a07b5126c12d5077d9a641a38c/tts_en_fastpitch_align.nemo.
[NeMo I 2024-11-03 14:53:52 cloud:64] Re-using file from: /home/antixk/.cache/torch/NeMo/NeMo_1.23.0/tts_en_fastpitch_align/b7d086a07b5126c12d5077d9a641a38c/tts_en_fastpitch_align.nemo
[NeMo I 2024-11-03 14:53:52 common:924] Instantiating model from pre-trained checkpoint


 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.
[NeMo W 2024-11-03 14:54:07 en_us_arpabet:66] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2024-11-03 14:54:07 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /ws/LJSpeech/nvidia_ljspeech_train_clean_ngc.json
      sample_rate: 22050
      sup_data_path: /raid/LJSpeech/supplementary
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
 

[NeMo I 2024-11-03 14:54:07 features:289] PADDING: 1


[NeMo W 2024-11-03 14:54:07 nemo_logging:349] /home/antixk/miniconda3/envs/trt/lib/python3.10/site-packages/nemo/core/connectors/save_restore_connector.py:571: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related

[NeMo I 2024-11-03 14:54:07 save_restore_connector:249] Model FastPitchModel was successfully restored from /home/antixk/.cache/torch/NeMo/NeMo_1.23.0/tts_en_fastpitch_align/b7d086a07b5126c12d5077d9a641a38c/tts_en_fastpitch_align.nemo.
[NeMo I 2024-11-03 14:54:07 cloud:58] Found existing object /home/antixk/.cache/torch/NeMo/NeMo_1.23.0/tts_en_hifitts_hifigan_ft_fastpitch/dcbdefa79d5e3587c93636d4f53260b8/tts_en_hifitts_hifigan_ft_fastpitch.nemo.
[NeMo I 2024-11-03 14:54:07 cloud:64] Re-using file from: /home/antixk/.cache/torch/NeMo/NeMo_1.23.0/tts_en_hifitts_hifigan_ft_fastpitch/dcbdefa79d5e3587c93636d4f53260b8/tts_en_hifitts_hifigan_ft_fastpitch.nemo
[NeMo I 2024-11-03 14:54:07 common:924] Instantiating model from pre-trained checkpoint


[NeMo W 2024-11-03 14:54:07 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.VocoderDataset
      manifest_filepath: /manifests/hifitts_fp_1000epoch_mels_train.json
      sample_rate: 44100
      n_segments: 16384
      max_duration: null
      min_duration: 0.75
      load_precomputed_mel: true
      hop_length: 512
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 100
      num_workers: 4
    
[NeMo W 2024-11-03 14:54:07 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.VocoderDataset


[NeMo I 2024-11-03 14:54:07 features:289] PADDING: 0
[NeMo I 2024-11-03 14:54:07 features:297] STFT using exact pad
[NeMo I 2024-11-03 14:54:07 features:289] PADDING: 0
[NeMo I 2024-11-03 14:54:07 features:297] STFT using exact pad


[NeMo W 2024-11-03 14:54:07 nemo_logging:349] /home/antixk/miniconda3/envs/trt/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
      WeightNorm.apply(module, name, dim)
    


[NeMo I 2024-11-03 14:54:08 save_restore_connector:249] Model HifiGanModel was successfully restored from /home/antixk/.cache/torch/NeMo/NeMo_1.23.0/tts_en_hifitts_hifigan_ft_fastpitch/dcbdefa79d5e3587c93636d4f53260b8/tts_en_hifitts_hifigan_ft_fastpitch.nemo.


In [12]:
def infer(spec_gen_model, vocoder_model, str_input):
    parser_model = spec_gen_model
    with torch.no_grad():
        parsed = parser_model.parse(str_input)
        gen_spec_kwargs = {}
        
        
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, **gen_spec_kwargs)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
        if audio_generator == "hifigan":
            audio = vocoder_model._bias_denoise(audio, spectrogram).squeeze(1)
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

In [37]:


fastpitch.export("pretrained_models/fastpitch.onnx",
                 input_example=fastpitch.input_example(),
                 onnx_opset_version=19, 
                 dynamic_axes={'text' : {0 : 'batch_size',
                                         1 : "seq_length",},
                                'pitch':{0:'batch_size', 1:'pitch_length'},
                                'pace':{0:'batch_size', 1:'pace_length'},
                                'mel': {0:'batch_size', 
                                        2: 'mel_length'}},)
                #  verbose=True)

[NeMo W 2024-11-03 15:04:53 nemo_logging:349] /home/antixk/miniconda3/envs/trt/lib/python3.10/site-packages/torch/onnx/utils.py:1963: UserWarning: Provided key mel for dynamic axes is not a valid input/output name
      warnings.warn(
    
[NeMo W 2024-11-03 15:04:53 nemo_logging:349] /home/antixk/miniconda3/envs/trt/lib/python3.10/site-packages/torch/onnx/_internal/jit_utils.py:308: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:178.)
      _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
    
[NeMo W 2024-11-03 15:04:53 nemo_logging:349] /home/antixk/miniconda3/envs/trt/lib/python3.10/site-packages/torch/onnx/utils.py:663: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/on

[NeMo I 2024-11-03 15:04:54 exportable:131] Successfully exported FastPitchModel to pretrained_models/fastpitch.onnx


(['pretrained_models/fastpitch.onnx'],
 ['nemo.collections.tts.models.fastpitch.FastPitchModel exported to ExportFormat.ONNX'])

In [ ]:
hifigan.export("pretrained_models/hifigan.onnx",
                input_example=hifigan.input_example(),
                onnx_opset_version=19, 
                dynamic_axes={'spec': {0:'batch_size', 
                                      2: 'mel_length'}},)

Removing weight norm...
Removing weight norm...
[NeMo I 2024-11-03 15:43:32 exportable:131] Successfully exported HifiGanModel to pretrained_models/hifigan.onnx


(['pretrained_models/hifigan.onnx'],
 ['nemo.collections.tts.models.hifigan.HifiGanModel exported to ExportFormat.ONNX'])

: 

In [ ]:
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
trt.init_libnvinfer_plugins(TRT_LOGGER, namespace="")

onnxfile = "pretrained_models/hifigan.onnx"
# Sanity check
onnx_model = onnx.load(onnxfile)
onnx.checker.check_model(onnx_model)
del onnx_model

major, minor, patch = trt.__version__.split('.')

logger.info(f"TensorRT Version: {major}.{minor}.{patch}")
EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH) #we have enabled the explicit Batch


with (trt.Builder(TRT_LOGGER) as builder, 
      builder.create_network() as network, 
      trt.OnnxParser(network, TRT_LOGGER) as parser, 
      builder.create_builder_config() as builder_config):
    builder_config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1<< 30)

    # if args.use_amp:
    #     logger.info("Using FP16 Precision")
    #     builder_config.set_flag(trt.BuilderFlag.FP16)

    logger.info("Parsing ONNX file.")
    with open(onnxfile, 'rb') as model:
        if not parser.parse(model.read()):
            for error in range(parser.num_errors):
                logger.error(parser.get_error(error))
            logger.error("Failed to parse ONNX file.")
            raise Exception("Failed to parse ONNX file.")

    # Set dynamic shapes
    logger.info("Setting optimization profile.")
    profile = builder.create_optimization_profile()
    # TODO: Set proper dynamic shapes
    profile.set_shape("mel", (1, 80, 661), (4, 80, 661), (8, 80,661))
    builder_config.add_optimization_profile(profile)
    builder_config.default_device_type = trt.DeviceType.GPU

    logger.info("Building TensorRT engine. This may take a few minutes.")
    serialized_engine = builder.build_serialized_network(network, builder_config)

    with open('pretrained_models/hifigan.plan', 'wb') as f:
        f.write(serialized_engine)
        logger.info("TensorRT engine saved to pretrained_models/hifigan.plan")

2024-11-03 14:52:00.219 | INFO     | __main__:<module>:12 - TensorRT Version: 10.6.0
2024-11-03 14:52:00.221 | INFO     | __main__:<module>:23 - Parsing ONNX file.
2024-11-03 14:52:00.273 | INFO     | __main__:<module>:32 - Setting optimization profile.
2024-11-03 14:52:00.274 | INFO     | __main__:<module>:39 - Building TensorRT engine. This may take a few minutes.
2024-11-03 14:52:22.926 | INFO     | __main__:<module>:44 - TensorRT engine saved to pretrained_models/hifigan.plan


In [ ]:
[(k, v.shape) for k,v in fastpitch.input_example()[0].items()]


TypeError: 'list' object is not callable

In [46]:
# Create ONNXRuntime Session for FastPitch
options = ort.SessionOptions()
options.enable_profiling=False
# options.log_severity=0# options.log_verbosity_level0


ort_session = ort.InferenceSession( 'pretrained_models/fastpitch.onnx',
    sess_options=options,
    providers=[  'CUDAExecutionProvider'])

# print(dir(ort_session))
# print(ort_session.get_inputs())
# print(ort_session.get_outputs())
io_binding = ort_session.io_binding()

encoded_text = fastpitch.parse("Yo,waddup!, how are you?").detach().cpu().numpy()

print(encoded_text.shape)
pace = np.array([[1.0]], dtype=np.float32)
pitch = np.array([[1.0]], dtype=np.float32)

encoded_text = ort.OrtValue.ortvalue_from_numpy(encoded_text)
pace = ort.OrtValue.ortvalue_from_numpy(pace)
pitch = ort.OrtValue.ortvalue_from_numpy(pitch)

# ortvalue.device_name()  # 'cpu'
# ortvalue.shape()        # shape of the numpy array X
# ortvalue.data_type()    # 'tensor(float)'
# ortvalue.is_tensor()    # 'True'
# np.array_equal(ortvalue.numpy(), X)  # 'True'

results = ort_session.run(None, input_feed= {"text": encoded_text, "pitch":pitch, "pace": pace})

mel = results[0]

print(mel.shape)


(1, 24)
(1, 80, 135)


2024-11-03 15:39:23.576740519 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 5 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2024-11-03 15:39:23.579105394 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-11-03 15:39:23.579112703 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


In [ ]:
class HostDeviceMem(object):
    def __init__(self, host_mem, device_mem):
        """Within this context, host_mom means the cpu memory and device means the GPU memory"""
        self.host = host_mem
        self.device = device_mem

    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)

    def __repr__(self):
        return self.__str__()
    
with open("pretrained_models/hifigan.plan", "rb") as f:
    serialized_engine = f.read()

engine = runtime.deserialize_cuda_engine(serialized_engine)
context = engine.create_execution_context()
inputs, outputs, bindings = [], [], []
stream = cuda.Stream()

for i in range(engine.num_io_tensors):
    tensor_name = engine.get_tensor_name(i)
    logger.info(f"Tensor:{tensor_name}, Shape:{engine.get_tensor_shape(tensor_name)}")


# class

def infer(mel: np.array):
    # https://github.com/NVIDIA/TensorRT/issues/4230
    # Actual shapes of the inputs
    input_shapes = mel.shape

    B = input_shapes[0]
    inputs = []
    outputs = []
    bindings = []
    context.set_input_shape("mel", mel.shape)

    for i in range(engine.num_io_tensors):
        tensor_name = engine.get_tensor_name(i)
        dtype = trt.nptype(engine.get_tensor_dtype(tensor_name))

        # Check if it's an input or output tensor
        if engine.get_tensor_mode(tensor_name) == trt.TensorIOMode.INPUT:
            shape = input_shapes  # Get the shape from the input shapes

            size = trt.volume(shape)
            host_mem = cuda.pagelocked_empty(size, dtype)
            device_mem = cuda.mem_alloc(host_mem.nbytes)
            inputs.append(HostDeviceMem(host_mem, device_mem))
            bindings.append(int(device_mem))
            np.copyto(inputs[-1].host, locals()[tensor_name].ravel())
        else:
            temp_shape = (B, *engine.get_tensor_shape(tensor_name)[1:])
            # temp_shape = (1,)  # Placeholder, adjust if necessary
            size = trt.volume(temp_shape)
            # print(temp_shape, size, dtype)
            host_mem = cuda.pagelocked_empty(size, dtype)
            device_mem = cuda.mem_alloc(host_mem.nbytes)
            outputs.append(HostDeviceMem(host_mem, device_mem))
            bindings.append(int(device_mem))

    # Transfer inputs to device
    for i in range(len(inputs)):
        cuda.memcpy_htod_async(inputs[i].device, inputs[i].host, stream)

    # Set tensor address for each input/output
    for i in range(engine.num_io_tensors):
        context.set_tensor_address(engine.get_tensor_name(i), bindings[i])

    context.execute_async_v3(stream.handle)

    # Transfer predictions back
    cuda.memcpy_dtoh_async(outputs[0].host, outputs[0].device, stream)

    # Synchronize the stream
    stream.synchronize()

    return outputs[0].host

def cleanup():
    for input_mem in inputs:
        input_mem.device.free()  # Free device memory for each input
    for output_mem in outputs:
        output_mem.device.free()  # Free device memory for each output



# Run inference
mel = np.random.rand(2, 80, 661).astype(np.float32)
audio = np.random.rand(1, 2, 16).astype(np.float32)
start_time = perf_counter()
output = infer(mel)
end_time = perf_counter()
# print time in milliseconds
print(f"Time taken:{(end_time - start_time) * 1000:.2f} ms")
print("Output shape:", output.shape)

print(output)

# Clean up memory after inference
cleanup()